<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/mip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>